In [1]:
import tensorflow as tf
import sys
import numpy as np
from configs import DEFINES

In [3]:
def make_lstm_cell(mode, hiddenSize, index) :
    cell = tf.nn.rnn_cell.BasicLSTMCell(hiddenSize, name="lstm"+str(index), state_is_tuple=False)
    if mode == tf.estimator.ModeKeys.TRAIN :
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=DEFINES.dropout_width)
        
    return cell

In [4]:
def Model(features, labels, mode, params) :
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    if params['embedding'] == True :
        initializer = tf.contrib.layers.xavier_initializer()
        embedding_encoder = tf.get_variable(name="embedding_encoder", shape=[params['vocabulary_length']], params=['embedding_size'], 
                                           dtype=tf.float32, initializer=initializer, trainable=True)
    else :
        embedding_encoder = tf.eye(num_rows=params['vocabulary_length'], dtype=tf.float32)
        embedding_encoder = tf.get_variable(name="embedding_encoder", initializer=embedding_encoder, trainable=False)
    
    embedding_encoder_batch = tf.nn.embedding_lookup(params=embedding_encoder, ids=features['input'])
    
    with tf.variable_scope('encoder_scope', reuse=tf.AUTO_REUSE) :
        if params['multilayer'] == True :
            encoder_cell_list = [make_lstm_cell(mode, params['hidden_size'], i) for i in range(params['layer_size'])]
            rnn_cell = tf.contrib.rnn.MultiRNNCell(encoder_cell_list, state_is_tuple=False)
        else :
            rnn_cell = make_lstm_cell(mode, params['hidden_size'], "")
            
        encoder_outputs, encoder_states = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_encoder_batch, dtype=tf.float32)